In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
%run ../../load_magic/lists.py
from bs4 import BeautifulSoup as bs
import networkx as nx
import re
import urllib.request
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 SequenceMatcher	 attempt_to_pickle	 bs	 
check_4_doubles	 check_for_typos	 csv	 encoding	 example_iterrows	 get_column_descriptions	 get_page_tables	 load_csv	 load_dataframes	 
load_object	 math	 nx	 os	 pd	 pickle	 re	 save_dataframes	 similar	 
store_objects	 sys	 time	 urllib	 


In [2]:

def get_page_soup(page_url):
    with urllib.request.urlopen(page_url) as response:
        page_html = response.read()
    page_soup = bs(page_html, 'html.parser')
    
    return page_soup

In [96]:

tables_url = 'https://factvsfitness.com/histamine-intolerance-food-list/'
hystamine_path = r'E:\low_hystamine_diet.html'
#page_soup = get_page_soup(tables_url)
tables_list = get_page_tables(hystamine_path)

[(0, (13, 5))]


In [107]:

#itertools.combinations(iterable, r)
with open(hystamine_path, 'r', encoding=ENCODING_TYPE) as myfile:
    html_str = myfile.read()
strong_li_list = re.split('<[^>]+data-th="([^"]+)">', html_str, 0)
strong_li_list = [re.sub(r'>\s+', '>', re.sub(r'\s+<', '<', s)) for s in strong_li_list[1:]]
n = len(strong_li_list)

In [126]:

rows_list = []
row_dict = {}
for i in range(0, n, 2):
    strong_li_str = strong_li_list[i+1]
    allowed_restricted = strong_li_list[i]
    if allowed_restricted == 'ALLOWED':
        row_dict['is_restricted'] = False
    elif allowed_restricted == 'RESTRICTED':
        row_dict['is_restricted'] = True
    for match_obj in re.finditer(r'<(strong|li)>([^<]+)</\1>', strong_li_str):
        #print(match_obj.group(1), match_obj.group(2))
        if match_obj.group(1) == 'strong':
            row_dict['food_group'] = match_obj.group(2)
        elif match_obj.group(1) == 'li':
            row_dict['food_name'] = match_obj.group(2)
            #print(row_dict)
            rows_list.append(row_dict.copy())
    #print(allowed_restricted, strong_li_str)
hystamine_df = pd.DataFrame(rows_list)
columns_list = ['is_restricted']
#[fn for fn in dir(hystamine_df) if 'dup' in fn.lower()]
hystamine_df.drop_duplicates(subset=columns_list)

,food_group,is_restricted,food_name
0,Vegetables,False,All fresh vegetables except those listed opposite
1,Vegetables,True,Eggplant


In [129]:

match_series = hystamine_df['is_restricted']
restricted_list = hystamine_df[match_series]['food_name'].tolist()
unrestricted_list = hystamine_df[~match_series]['food_name'].tolist()

In [130]:

restricted_list

['Eggplant', 'Pumpkin', 'Sauerkraut', 'Spinach', 'Tomato', 'Avocado', 'Olives', 'Pickled', 'vegetables', 'Citrus fruits', 'Banana', 'Cherry', 'Cranberry', 'Currant', 'Date', 'Loganberry', 'Nectarine', 'Orange', 'Papaya', 'Peach', 'Pineapple', 'Prunes', 'Plums', 'Raisins', 'Raspberries', 'Strawberries', 'Fruit dishes,jams, juices made with restricted ingredients', 'All fish (unless fresh caught)', 'All shellfish', 'Leftover/refrigerated meats', 'Processed, cured or smoked meats', 'Raw egg whites (as in some eggnog, hollandaise sauce, milk shakes)', 'All fats and oils with colours or preservatives', 'Prepared gravy', 'Commercial salad dressings', 'Hydrolyzed lecithin', 'Anise', 'Cinnamon', 'Cloves', 'Curry powder', 'Paprika/cayenne', 'Nutmeg', 'Seasoning packets with restricted ingredients', 'Foods labeled “with spices”', 'Walnuts', 'Cashews', 'Soy', 'Lentils', 'Beans', 'Peanuts', 'Foods cooked in oils containing hydrolyzed lecithin, BHA or BHT Baking Mixes', 'Dry dessert mixes', 'All ch

In [131]:

unrestricted_list

['All fresh vegetables except those listed opposite', 'Apple', 'Cantaloupe', 'Figs', 'Grapefruit', 'Grapes', 'Honeydew', 'Mango', 'Pear', 'Rhubarb', 'Lychee', 'Persimmon', 'Blackberries', 'Blueberries', 'Watermelon', 'Coconut', 'All fresh meat and poultry. Must be bought and cooked fresh.', 'Plain eggs (ensure whites are fully cooked)', 'Pure butter', 'All vegetable oils and oils of allowed', 'foods which contain no additives', 'Coconut oil', 'Meat drippings and fat', 'Homemade gravy', 'Homemade salad dressings with allowed ingredients', 'All fresh herbs and spices except those listed opposite', 'All nuts should be eliminated for the first two weeks on the diet. After this period, nuts aside from those listed opposite may be reintroduced individually to examine tolerance. Begin with macadamias and chestnuts.', 'All legumes should be eliminated for the first two weeks on the diet. After this period, legumes aside from those listed opposite may be reintroduced individually to examine tol

In [97]:

tables_list[0]

,FOOD GROUP,ALLOWED,ALLOWED.1,RESTRICTED,RESTRICTED.1
0,Vegetables,All fresh vegetables except those listed opposite,All fresh vegetables except those listed opposite,"Eggplant, Pumpkin, Sauerkraut, Spinach, Tomato","Avocado, Olives, Pickled, vegetables"
1,Fruits,"Apple, Cantaloupe, Figs, Grapefruit, Grapes, H...","Rhubarb, Lychee, Persimmon, Blackberries, Blue...","Citrus fruits, Banana, Cherry, Cranberry, Curr...","Pineapple, Prunes, Plums, Raisins, Raspberries..."
2,"Meat, Poultry and Fish",All fresh meat and poultry. Must be bought and...,All fresh meat and poultry. Must be bought and...,"All fish (unless fresh caught), All shellfish,...","All fish (unless fresh caught), All shellfish,..."
3,Eggs,Plain eggs (ensure whites are fully cooked),Plain eggs (ensure whites are fully cooked),"Raw egg whites (as in some eggnog, hollandaise...","Raw egg whites (as in some eggnog, hollandaise..."
4,Fats and Oils,"Pure butter, All vegetable oils and oils of al...","Pure butter, All vegetable oils and oils of al...",All fats and oils with colours or preservative...,All fats and oils with colours or preservative...
5,Spices and Herbs,All fresh herbs and spices except those listed...,All fresh herbs and spices except those listed...,"Anise, Cinnamon, Cloves, Curry powder, Paprika...","Anise, Cinnamon, Cloves, Curry powder, Paprika..."
6,Nuts and seeds,All nuts should be eliminated for the first tw...,All nuts should be eliminated for the first tw...,"Walnuts, Cashews","Walnuts, Cashews"
7,Legumes*,All legumes should be eliminated for the first...,All legumes should be eliminated for the first...,"Soy, Lentils, Beans, Peanuts","Soy, Lentils, Beans, Peanuts"
8,Breads and Cereals*,Total of Â½ cup per day maximum of the followi...,Total of Â½ cup per day maximum of the followi...,Foods cooked in oils containing hydrolyzed lec...,Foods cooked in oils containing hydrolyzed lec...
9,Milk and Dairy*,"After two weeks on this diet, plain milk can b...","After two weeks on this diet, plain milk can b...","All cheese, All yogurt (unless specifically cu...","All cheese, All yogurt (unless specifically cu..."
